In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from code.survey_processor_full import *
from code.fc_cluster import *

import matplotlib.pylab as plt
import seaborn as sns

%matplotlib inline


In [2]:
sv = survey('data/survey2/')

In [3]:
sv.num_processor()

In [4]:
df = sv.fin_data

In [5]:
dfsmall = pd.read_pickle('data/df_id.pkl')

In [6]:
cols = list(dfsmall.columns)

In [7]:
df = df[cols[:-1]]

In [8]:
cl_df = pd.DataFrame(np.column_stack((df,dfsmall.cluster)),columns=cols)

In [9]:
cl_df = cl_df.astype('float')

In [10]:
heat_df = cl_df.groupby('cluster').agg(np.median)

In [ ]:
sns.clustermap(heat_df, metric="cosine")